In [ ]:
# torch with cuda
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from ultralytics import YOLO
import torch
import os

# Load a model
# model = YOLO("best.onnx")
model = YOLO("yolo11n.pt")

path = os.path.join(os.path.abspath("."),"License Plate Recognition.v6i.yolov11.zip")

# Train the model
train_results = model.train(
    data=path,  # path to dataset YAML
    epochs=100,  # number of training epochs
    # workers=100,
    # patience=20,
    imgsz=640,  # training image size
    device="cuda" if torch.cuda.is_available() else "cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

# Evaluate model performance on the validation set
metrics = model.val()

# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model

In [ ]:
import cv2
from matplotlib import pyplot as plt
from plate_recognition import NumberPlateRecognition

rec = NumberPlateRecognition()

def run(img):
    rec.analyze(img)

# Perform object detection on an image
img = cv2.imread("example.jpg")
run(img)
img = cv2.flip(img, 0)
run(img)
img = cv2.flip(img, 1)
run(img)
img = cv2.flip(img, 0)
run(img)

In [3]:
from safe_video import NumberPlateRecognition
import cv2

rec = NumberPlateRecognition()
img = cv2.imread("example.jpg")
img = img[:, :, ::-1]
analyzed_img = rec.analyze(img)
result = rec.blur_image(img, analyzed_img.boxes)

plt.imshow(img)
plt.show()
plt.imshow(result)
plt.show()
# original = Image.fromarray(img, 'RGB')
# result2 = Image.fromarray(result, 'RGB')

Loading c:\Ordner\Uni\Master\24-Winter\Bildverarbeitung\NumberPlateRecognition\models\first10ktrain\weights\best.onnx for ONNX Runtime inference...
Preferring ONNX Runtime TensorrtExecutionProvider
*************** EP Error ***************
EP Error D:\a\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:507 onnxruntime::python::RegisterTensorRTPluginsAsCustomOps Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************

0: 640x640 4 License_Plates, 10.0ms
Speed: 79.0ms preprocess, 10.0ms inference, 155.0ms postprocess per image at shape (1, 3, 640, 640)
boxes: [[     431.77      437.72      476.63      453.35]
 [     626.27      432.12      674.95      446.11]
 [        129